In [13]:
import pandas as pd
from functools import reduce

In [14]:
twitter_dir = 'data/in/twitter/'
news_dir = 'data/in/news/'
stock_dir = 'data/out/'

In [23]:
influencers = ['BarackObama','cnnbrk', 'KimKardashian']
sources = ['New York Times', 'CNN', 'Washington Post']
outs = ['VIX_Future']

In [65]:
twitter_important_columns = ['Date','sentiment_score']

In [16]:
first_date = pd.to_datetime('1/1/2015')
last_date = pd.to_datetime('1/1/2017')

In [66]:
dfs_twitter = []
for influencer in influencers:
    df = pd.read_csv(twitter_dir+influencer+'.csv', parse_dates=['Date'])
    df['Date']= pd.to_datetime(df['Date'].dt.normalize())
    df = df[df['Date']<last_date]
    df = df[df['Date']>first_date]
    df = df.groupby('Date').sum()
    df = df[twitter_important_columns]
    df = df.rename(columns={'sentiment_score':influencer+'_sentiment_score'})
    dfs_twitter.append(df)

In [67]:
dfs_news = []
for source in sources:
    df = pd.read_csv(news_dir+source+'.csv', parse_dates=['Date'])
    df['Date']= pd.to_datetime(df['Date'].dt.normalize())
    df = df[df['Date']<last_date]
    df = df[df['Date']>first_date]
    df = df.groupby('Date').sum()
    df = df.rename(columns={'sentiment_score':source+'_sentiment_score'})
    dfs_news.append(df)

In [68]:
dfs_out = []
for out in outs:
    df = pd.read_csv(stock_dir+out+'.csv', parse_dates=['Date'])
    df['Date']= pd.to_datetime(df['Date'].dt.normalize())
    df = df[df['Date']<last_date]
    df = df[df['Date']>first_date]
    dfs_out.append(df)

In [69]:
dfs_twitter[1]

,cnnbrk_sentiment_score
Date,
2016-01-08,-2.1748
2016-01-09,-0.3230
2016-01-10,-0.8920
2016-01-11,-1.8569
2016-01-12,0.0023
...,...
2016-12-27,-1.1141
2016-12-28,-1.7633
2016-12-29,-0.2428


In [70]:
df_twitter = reduce(lambda  left,right: pd.merge(left,right,on=['Date'],
                                            how='outer'), dfs_twitter)

In [71]:
df_news = reduce(lambda  left,right: pd.merge(left,right,on=['Date'],
                                            how='outer'), dfs_news)

In [72]:
df_out = reduce(lambda  left,right: pd.merge(left,right,on=['Date'],
                                            how='outer'), dfs_out)

In [73]:
predict_dates = df_out['Date']

In [74]:
#FOR NOW ONLY DELETE THE WEEKENDS
df_twitter = pd.merge(predict_dates, df_twitter, on=["Date"], how='left')
df_news = pd.merge(predict_dates, df_news, on=["Date"], how='left')

In [75]:
df = pd.merge(df_out, df_twitter, how='left', on='Date')
df2 = pd.merge(df_out, df_news, how='left', on='Date')


In [76]:
df.corr()

,Price,Change,Lag 2 Price,2 Day Change,2 Day Direction,2 Day Significant,BarackObama_sentiment_score,cnnbrk_sentiment_score,KimKardashian_sentiment_score
Price,1.000000,0.200055,0.873128,-0.215268,-0.214931,-0.211339,0.015731,0.052405,0.091219
Change,0.200055,1.000000,0.173408,-0.044397,-0.093826,-0.096219,0.021521,0.043728,0.047395
Lag 2 Price,0.873128,0.173408,1.000000,0.274198,0.134506,0.151959,0.008689,0.141614,0.043482
2 Day Change,-0.215268,-0.044397,0.274198,1.000000,0.719067,0.746575,-0.018619,0.169578,-0.080551
2 Day Direction,-0.214931,-0.093826,0.134506,0.719067,1.000000,0.956997,0.044994,0.163398,-0.049362
2 Day Significant,-0.211339,-0.096219,0.151959,0.746575,0.956997,1.000000,0.043486,0.175315,-0.050814
BarackObama_sentiment_score,0.015731,0.021521,0.008689,-0.018619,0.044994,0.043486,1.000000,-0.006240,0.069529
cnnbrk_sentiment_score,0.052405,0.043728,0.141614,0.169578,0.163398,0.175315,-0.006240,1.000000,0.198882
KimKardashian_sentiment_score,0.091219,0.047395,0.043482,-0.080551,-0.049362,-0.050814,0.069529,0.198882,1.000000


In [64]:
df2.corr()

,Price,Change,Lag 2 Price,2 Day Change,2 Day Direction,2 Day Significant,New York Times_sentiment_score,CNN_sentiment_score,Washington Post_sentiment_score
Price,1.000000,0.200055,0.873128,-0.215268,-0.214931,-0.211339,0.010019,0.058390,0.091169
Change,0.200055,1.000000,0.173408,-0.044397,-0.093826,-0.096219,-0.065055,0.066248,-0.062777
Lag 2 Price,0.873128,0.173408,1.000000,0.274198,0.134506,0.151959,-0.069311,0.108935,0.073697
2 Day Change,-0.215268,-0.044397,0.274198,1.000000,0.719067,0.746575,-0.107881,0.080929,-0.036708
2 Day Direction,-0.214931,-0.093826,0.134506,0.719067,1.000000,0.956997,-0.101791,0.068496,-0.012672
2 Day Significant,-0.211339,-0.096219,0.151959,0.746575,0.956997,1.000000,-0.064139,0.089364,-0.013954
New York Times_sentiment_score,0.010019,-0.065055,-0.069311,-0.107881,-0.101791,-0.064139,1.000000,-0.042060,0.021621
CNN_sentiment_score,0.058390,0.066248,0.108935,0.080929,0.068496,0.089364,-0.042060,1.000000,0.224250
Washington Post_sentiment_score,0.091169,-0.062777,0.073697,-0.036708,-0.012672,-0.013954,0.021621,0.224250,1.000000


In [ ]:
#OLD

In [235]:
data_VIX = pd.read_csv('VIX.csv', parse_dates=['Date'])
data_Obama = pd.read_csv('BarackObama.csv', parse_dates=['Date'])
data_NYT = pd.read_csv('New York Times.csv', parse_dates=['Date'])

In [236]:
data_Obama['Date']= pd.to_datetime(data_Obama['Date'].dt.normalize())
data_VIX['Date']= pd.to_datetime(data_VIX['Date'].dt.normalize())
data_NYT['Date']= pd.to_datetime(data_NYT['Date']).dt.normalize()

In [237]:
data_NYT

,title,author,Date,sentiment_score
0,37 Who Saw Murder Didn't Call the Police - The...,NaN,2014-05-08,-0.6908
1,The Pillow Cure - The New York Times,Penelope Green,2015-03-26,0.0000
2,John Ashbery: By the Book - The New York Times,NaN,2015-05-13,0.0000
3,"Drink to Your Health (in Moderation), the Scie...",Aaron E. Carroll,2015-12-22,0.0000
4,Simple Rules for Healthy Eating - The New York...,Aaron E. Carroll,2015-12-28,0.4019
...,...,...,...,...
30252,How a Trump Adviser Repeatedly Sought a Meetin...,Troy Griggs,NaT,0.0000
30253,"Satellite Images Show 1,800 Buildings Destroye...",Anjali Singhvi,NaT,-0.6808
30254,Trump Could Save More Than $1 Billion Under Hi...,Jesse Drucker,NaT,0.4939
30255,Opinion | The Republican’s Guide to Presidenti...,The Editorial Board,NaT,0.0000


In [238]:
data_Obama

,Date,author,content,number_of_likes,number_of_shares,sentiment_score
0,2014-01-08,BarackObama,Make #53 special for #44: http://t.co/leGTnedv...,1512,726,0.4019
1,2014-01-08,BarackObama,"Job growth has exceeded 200,000 for the sixth ...",816,556,0.4854
2,2014-01-08,BarackObama,Happening now: President Obama delivers a stat...,491,288,0.0000
3,2014-01-08,BarackObama,"""That's the longest streak of private sector j...",804,619,0.6249
4,2014-01-08,BarackObama,"""The fact is we could be much further along .....",1505,1260,0.0000
...,...,...,...,...,...,...
2858,2016-12-10,BarackObama,Let's leave our children and grandchildren a p...,12072,3692,-0.0516
2859,2016-12-10,BarackObama,"Last week, more than 190 countries reached a l...",3283,855,0.1779
2860,2016-12-10,BarackObama,Support the bold steps we're taking against cl...,1979,496,0.7783
2861,2016-12-10,BarackObama,LIVE: President Obama is speaking at a recepti...,1931,468,0.5719


In [240]:
data_Obama = data_Obama[data_Obama['Date']<last_date]
data_Obama = data_Obama[data_Obama['Date']>first_date]
data_VIX = data_VIX[data_VIX['Date']<last_date]
data_VIX = data_VIX[data_VIX['Date']>first_date]
data_NYT = data_NYT[data_NYT['Date']<last_date]
data_NYT = data_NYT[data_NYT['Date']>first_date]

In [241]:
data_VIX['Lag 2 Price'] = data_VIX['Price'].shift(-2)
data_VIX['2 Day Change'] = (data_VIX['Lag 2 Price'] - data_VIX['Price'])/data_VIX['Price']

In [242]:
data_VIX

,Date,Price,Change,Lag 2 Price,2 Day Change
3773,2015-01-02,17.76,0.116279,20.33,0.144707
3774,2015-01-05,19.19,0.080518,20.15,0.050026
3775,2015-01-06,20.33,0.059406,17.93,-0.118052
3776,2015-01-07,20.15,-0.008854,16.44,-0.184119
3777,2015-01-08,17.93,-0.110174,18.02,0.005020
...,...,...,...,...,...
4272,2016-12-23,11.38,0.005300,11.89,0.044815
4273,2016-12-27,12.26,0.077329,13.15,0.072594
4274,2016-12-28,11.89,-0.030179,13.20,0.110177
4275,2016-12-29,13.15,0.105971,NaN,NaN


In [243]:
data_NYT

,title,author,Date,sentiment_score
1,The Pillow Cure - The New York Times,Penelope Green,2015-03-26,0.0000
2,John Ashbery: By the Book - The New York Times,NaN,2015-05-13,0.0000
3,"Drink to Your Health (in Moderation), the Scie...",Aaron E. Carroll,2015-12-22,0.0000
4,Simple Rules for Healthy Eating - The New York...,Aaron E. Carroll,2015-12-28,0.4019
5,The Secret to Good Toast? It’s Your Freezer - ...,Emily Weinstein,2016-02-27,0.4404
...,...,...,...,...
3636,"Whitney Houston, Cher, Tina Turner and Donna S...",Erik Piepenburg,2016-12-30,0.0000
3637,"First, a Mixtape. Then a Romance. - The New Yo...",Katherine Rosman,2016-12-31,0.5574
3638,Calling on Angels While Enduring the Trials of...,Andy Newman,2016-12-31,0.0000
3639,House Republicans Fret About Winning Their Hea...,Carl Hulse,2016-12-31,0.7650


In [244]:
df = data_Obama.groupby('Date').sum()

In [245]:
dfNYT = dfNYT.groupby('Date').sum()

In [247]:
df2 = pd.merge(data_VIX, df, how='left', on='Date', suffixes = ('VIX', 'Obama'))
df2 = pd.merge(df2, dfNYT, how='left', on='Date', suffixes = ('', 'NYT'))

In [248]:
df2

,Date,Price,Change,Lag 2 Price,2 Day Change,number_of_likes,number_of_shares,sentiment_score,Score
0,2015-01-02,17.76,0.116279,20.33,0.144707,NaN,NaN,NaN,NaN
1,2015-01-05,19.19,0.080518,20.15,0.050026,4182.0,3099.0,1.2205,NaN
2,2015-01-06,20.33,0.059406,17.93,-0.118052,87467.0,57145.0,0.6436,NaN
3,2015-01-07,20.15,-0.008854,16.44,-0.184119,16168.0,6205.0,5.6336,NaN
4,2015-01-08,17.93,-0.110174,18.02,0.005020,9612.0,4116.0,1.1121,NaN
...,...,...,...,...,...,...,...,...,...
499,2016-12-23,11.38,0.005300,11.89,0.044815,NaN,NaN,NaN,0.3667
500,2016-12-27,12.26,0.077329,13.15,0.072594,NaN,NaN,NaN,-0.9913
501,2016-12-28,11.89,-0.030179,13.20,0.110177,NaN,NaN,NaN,2.9580
502,2016-12-29,13.15,0.105971,NaN,NaN,NaN,NaN,NaN,0.9901


In [223]:
df2.corr()

,Price,Change,Lag 2 Price,2 Day Change,number_of_likes,number_of_shares,Score
Price,1.000000,0.200055,0.872895,-0.215229,-0.016306,-0.017743,0.010019
Change,0.200055,1.000000,0.175280,-0.045860,0.031180,0.039245,-0.065055
Lag 2 Price,0.872895,0.175280,1.000000,0.274683,0.023407,0.019951,-0.066094
2 Day Change,-0.215229,-0.045860,0.274683,1.000000,0.082644,0.083975,-0.111391
number_of_likes,-0.016306,0.031180,0.023407,0.082644,1.000000,0.953408,0.098481
number_of_shares,-0.017743,0.039245,0.019951,0.083975,0.953408,1.000000,0.125824
Score,0.010019,-0.065055,-0.066094,-0.111391,0.098481,0.125824,1.000000
